# Descriptor & Feature Matrix Creation (for LightGBM) @

## Import and setup 

In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors, AllChem
from rdkit.Chem import rdMolDescriptors
import os
from tqdm import tqdm
import warnings

warnings.filterwarnings("ignore")
tqdm.pandas()

# Paths
DATA_PATH = "tox21_lightgb_pipeline/Data_v6/processed/tox21.csv"
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed/"

## Load Dataset

In [2]:
df = pd.read_csv(DATA_PATH)

# Check structure
assert df.shape[1] == 14, "Expected 12 labels + mol_id + smiles"
label_cols = df.columns[:12].tolist()
meta_cols = ["mol_id", "smiles"]

print(f"Labels: {label_cols}")
df.head()


Labels: ['NR-AR', 'NR-AR-LBD', 'NR-AhR', 'NR-Aromatase', 'NR-ER', 'NR-ER-LBD', 'NR-PPAR-gamma', 'SR-ARE', 'SR-ATAD5', 'SR-HSE', 'SR-MMP', 'SR-p53']


,NR-AR,NR-AR-LBD,NR-AhR,NR-Aromatase,NR-ER,NR-ER-LBD,NR-PPAR-gamma,SR-ARE,SR-ATAD5,SR-HSE,SR-MMP,SR-p53,mol_id,smiles
0,0.0,0.0,1.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,TOX3021,CCOc1ccc2nc(S(N)(=O)=O)sc2c1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3020,CCN1C(=O)NC(c2ccccc2)C1=O
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,TOX3024,CC[C@]1(O)CC[C@H]2[C@@H]3CCC4=CCCC[C@@H]4[C@H]...
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,0.0,TOX3027,CCCN(CC)C(CC)C(=O)Nc1c(C)cccc1C
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,TOX20800,CC(O)(P(=O)(O)O)P(=O)(O)O


## RDKit Descriptor Function (PhysChem + ECFP4)

In [3]:
def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    
    # PhysChem descriptors
    physchem = {
        'MolWt': Descriptors.MolWt(mol),
        'LogP': Descriptors.MolLogP(mol),
        'TPSA': Descriptors.TPSA(mol),
        'NumRotatableBonds': Descriptors.NumRotatableBonds(mol),
        'NumHDonors': Descriptors.NumHDonors(mol),
        'NumHAcceptors': Descriptors.NumHAcceptors(mol),
        'RingCount': Descriptors.RingCount(mol),
    }

    # ECFP4 fingerprint (bit vector)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, radius=2, nBits=2048)
    ecfp_bits = np.array(fp)
    ecfp = {f'ECFP4_{i}': int(bit) for i, bit in enumerate(ecfp_bits)}
    
    return {**physchem, **ecfp}


## Apply Descriptor Function

In [4]:
# Apply descriptor function to all molecules
desc_data = []
invalid_idx = []

for idx, smi in tqdm(enumerate(df['smiles']), total=len(df)):
    result = compute_descriptors(smi)
    if result is None:
        invalid_idx.append(idx)
    else:
        desc_data.append(result)

# Drop invalid rows
print(f"Dropping {len(invalid_idx)} invalid SMILES rows.")
df_clean = df.drop(index=invalid_idx).reset_index(drop=True)
desc_df = pd.DataFrame(desc_data).reset_index(drop=True)


100%|██████████| 7831/7831 [00:09<00:00, 834.72it/s]


Dropping 0 invalid SMILES rows.


## Handle Label NaNs which is in our class columns

In [5]:
# Create a mask (same shape as Y) to indicate which entries are valid
label_mask = ~df_clean[label_cols].isna().values  # True where label is NOT NaN

# Fill NaNs in labels with -1 (temporarily) — we'll ignore these during training
Y = df_clean[label_cols].fillna(-1).values.astype(np.int32)


## Prepare Final X, Y Matrices

In [6]:
X = desc_df.values.astype(np.float32)
smiles = df_clean["smiles"].tolist()
mol_ids = df_clean["mol_id"].tolist()

print(f"Final feature matrix shape: {X.shape}")
print(f"Label matrix shape: {Y.shape}")
print(f"Missing labels per class: {np.sum(~label_mask, axis=0)}")


Final feature matrix shape: (7831, 2055)
Label matrix shape: (7831, 12)
Missing labels per class: [ 566 1073 1282 2010 1638  876 1381 1999  759 1364 2021 1057]


## Save Everything for Later Use

In [7]:
np.save(os.path.join(SAVE_DIR, "X.npy"), X)
np.save(os.path.join(SAVE_DIR, "Y.npy"), Y)
np.save(os.path.join(SAVE_DIR, "label_mask.npy"), label_mask)
np.save(os.path.join(SAVE_DIR, "smiles_list.npy"), smiles)
np.save(os.path.join(SAVE_DIR, "mol_id_list.npy"), mol_ids)

# Save feature names too
with open(os.path.join(SAVE_DIR, "feature_names.txt"), "w") as f:
    f.writelines([col + "\n" for col in desc_df.columns])

print("✅ Saved: X.npy, Y.npy, label_mask.npy, smiles_list.npy, mol_id_list.npy, feature_names.txt")


✅ Saved: X.npy, Y.npy, label_mask.npy, smiles_list.npy, mol_id_list.npy, feature_names.txt


# New inputs and start from here if you have the descriptors and the other inputs @@

## Data Loading & Preparation

In [8]:
# STEP 1: Load preprocessed data
import numpy as np
import pandas as pd

DATA_PATH = "tox21_lightgb_pipeline/Data_v6/processed"

# Load features and labels
X = np.load(f"{DATA_PATH}/X.npy")
Y = np.load(f"{DATA_PATH}/Y.npy")

# Load auxiliary data
label_mask = np.load(f"{DATA_PATH}/label_mask.npy")
mol_ids = np.load(f"{DATA_PATH}/mol_id_list.npy", allow_pickle=True)
smiles_list = np.load(f"{DATA_PATH}/smiles_list.npy", allow_pickle=True)

# Load feature names
with open(f"{DATA_PATH}/feature_names.txt") as f:
    feature_names = f.read().splitlines()

# Define label column names (Tox21 labels)
label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

# Check dimensions
print(f"✅ Loaded X shape: {X.shape}")
print(f"✅ Loaded Y shape: {Y.shape}")
print(f"✅ Label mask shape: {label_mask.shape}")
print(f"✅ # Feature names: {len(feature_names)}")
print(f"✅ # SMILES: {len(smiles_list)}, # Mol IDs: {len(mol_ids)}")


✅ Loaded X shape: (7831, 2055)
✅ Loaded Y shape: (7831, 12)
✅ Label mask shape: (7831, 12)
✅ # Feature names: 2055
✅ # SMILES: 7831, # Mol IDs: 7831


# train/val/test splits + model training @@@

## Imports & Reload Feature/Label Data

In [9]:
import numpy as np
import pandas as pd
import joblib
import os
import shap
import lightgbm as lgb
import xgboost as xgb
import optuna

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.feature_selection import SelectFromModel
from lightgbm import early_stopping, log_evaluation

# Set paths
MODEL_PATH = "tox21_lightgb_pipeline/models/v6"
DATA_PATH = "tox21_lightgb_pipeline/Data_v6/processed"
os.makedirs(MODEL_PATH, exist_ok=True)

# Load data
X = np.load(os.path.join(DATA_PATH, "X.npy"))
Y = np.load(os.path.join(DATA_PATH, "Y.npy"))
label_mask = np.load(os.path.join(DATA_PATH, "label_mask.npy"))
feature_names = [line.strip() for line in open(os.path.join(DATA_PATH, "feature_names.txt"))]

# Define class labels
label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]


## Training Loop (Feature Selection + Optuna + Model Training)

In [10]:
results = []
feature_mask_dict = {}
threshold_dict = {}

for i, label_name in enumerate(label_cols):
    try:
        print(f"\n📍 Training for label {i+1}/{len(label_cols)}: {label_name}")

        # Filter valid samples
        valid_rows = label_mask[:, i]
        X_valid = X[valid_rows]
        y_valid = Y[valid_rows, i]

        # Train/val/test split
        X_train, X_temp, y_train, y_temp = train_test_split(
            X_valid, y_valid, test_size=0.2, random_state=42, stratify=y_valid
        )
        X_val, X_test, y_val, y_test = train_test_split(
            X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
        )

        # Feature selection using median importance
        selector_model = lgb.LGBMClassifier(n_estimators=200, random_state=42)
        selector_model.fit(X_train, y_train)
        selector = SelectFromModel(selector_model, threshold="median", prefit=True)
        X_train_fs = selector.transform(X_train)
        X_val_fs = selector.transform(X_val)
        X_test_fs = selector.transform(X_test)
        kept_features = selector.get_support(indices=True)
        feature_mask_dict[label_name] = kept_features
        print(f"→ Reduced to {X_train_fs.shape[1]} features.")

        # Hyperparameter tuning using Optuna
        dtrain = lgb.Dataset(X_train_fs, label=y_train)
        dval = lgb.Dataset(X_val_fs, label=y_val)

        def objective(trial):
            params = {
                "objective": "binary",
                "metric": "auc",
                "boosting_type": "gbdt",
                "verbosity": -1,
                "feature_pre_filter": False,
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
                "num_leaves": trial.suggest_int("num_leaves", 20, 80),
                "max_depth": trial.suggest_int("max_depth", 3, 12),
                "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
                "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
                "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
                "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 1.0),
                "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 1.0),
            }
            model = lgb.train(
                params.copy(),
                dtrain,
                valid_sets=[dval],
                callbacks=[
                    lgb.early_stopping(stopping_rounds=30),
                    lgb.log_evaluation(period=0)
                ]
            )
            preds = model.predict(X_val_fs)
            return roc_auc_score(y_val, preds)

        study = optuna.create_study(direction="maximize")
        study.optimize(objective, n_trials=30)
        best_params = study.best_params
        best_params.update({
            "objective": "binary",
            "metric": "auc",
            "boosting_type": "gbdt",
            "verbosity": -1
        })

        # Train best LightGBM
        clf_lgb = lgb.LGBMClassifier(**best_params, n_estimators=500, random_state=42)
        clf_lgb.fit(X_train_fs, y_train)

        # Train XGBoost
        clf_xgb = xgb.XGBClassifier(
            objective="binary:logistic",
            eval_metric="auc",
            n_estimators=500,
            use_label_encoder=False,
            random_state=42
        )
        clf_xgb.fit(X_train_fs, y_train)

        # Ensemble prediction on val
        val_prob_lgb = clf_lgb.predict_proba(X_val_fs)[:, 1]
        val_prob_xgb = clf_xgb.predict_proba(X_val_fs)[:, 1]
        val_prob_avg = (val_prob_lgb + val_prob_xgb) / 2

        # Threshold tuning
        thresholds = np.linspace(0.1, 0.9, 81)
        best_f1, best_thresh = 0, 0.5
        for t in thresholds:
            preds = (val_prob_avg >= t).astype(int)
            f1 = f1_score(y_val, preds)
            if f1 > best_f1:
                best_f1 = f1
                best_thresh = t

        threshold_dict[label_name] = best_thresh

        # Final test prediction
        test_prob_avg = (
            clf_lgb.predict_proba(X_test_fs)[:, 1] +
            clf_xgb.predict_proba(X_test_fs)[:, 1]
        ) / 2
        test_preds = (test_prob_avg >= best_thresh).astype(int)

        auc = roc_auc_score(y_test, test_prob_avg)
        acc = accuracy_score(y_test, test_preds)

        print(f"✅ {label_name} - AUC: {auc:.4f}, Accuracy: {acc:.4f}, Threshold: {best_thresh:.2f}")

        # Save models
        joblib.dump(clf_lgb, os.path.join(MODEL_PATH, f"{label_name}_lgb.pkl"))
        joblib.dump(clf_xgb, os.path.join(MODEL_PATH, f"{label_name}_xgb.pkl"))

        # Save result row
        results.append({
            "label": label_name,
            "auc": round(auc, 4),
            "accuracy": round(acc, 4),
            "n_train": len(y_train),
            "n_val": len(y_val),
            "n_test": len(y_test),
            "best_threshold": round(best_thresh, 2),
            "num_features": X_train_fs.shape[1]
        })

    except Exception as e:
        print(f"❌ Error while training {label_name}: {e}")



📍 Training for label 1/12: NR-AR
[LightGBM] [Info] Number of positive: 247, number of negative: 5565
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012984 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3983
[LightGBM] [Info] Number of data points in the train set: 5812, number of used features: 1567
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.042498 -> initscore=-3.114864
[LightGBM] [Info] Start training from score -3.114864


[I 2025-08-04 22:21:09,144] A new study created in memory with name: no-name-35acbcf7-3865-406a-916c-25ffe1f3b90a
[I 2025-08-04 22:21:09,310] Trial 0 finished with value: 0.7610350429333952 and parameters: {'learning_rate': 0.1534197919632927, 'num_leaves': 40, 'max_depth': 4, 'min_child_samples': 31, 'feature_fraction': 0.9685686370537981, 'bagging_fraction': 0.8306206305067867, 'lambda_l1': 0.03152263434474123, 'lambda_l2': 0.38320730831888317}. Best is trial 0 with value: 0.7610350429333952.


→ Reduced to 2055 features.
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[6]	valid_0's auc: 0.761035
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 22:21:09,404] Trial 1 finished with value: 0.7850545370155488 and parameters: {'learning_rate': 0.19218617318152445, 'num_leaves': 38, 'max_depth': 5, 'min_child_samples': 49, 'feature_fraction': 0.9458254589252173, 'bagging_fraction': 0.7524662653807541, 'lambda_l1': 0.7812170131764526, 'lambda_l2': 0.9012491966843506}. Best is trial 1 with value: 0.7850545370155488.
[I 2025-08-04 22:21:09,484] Trial 2 finished with value: 0.7448828034346716 and parameters: {'learning_rate': 0.11949354828721094, 'num_leaves': 79, 'max_depth': 7, 'min_child_samples': 48, 'feature_fraction': 0.9868447509700327, 'bagging_fraction': 0.9516318913341666, 'lambda_l1': 0.5183383912422327, 'lambda_l2': 0.6980586601948087}. Best is trial 1 with value: 0.7850545370155488.
[I 2025-08-04 22:21:09,564] Trial 3 finished with value: 0.7753074959387329 and parameters: {'learning_rate': 0.051812051384522086, 'num_leaves': 47, 'max_depth': 3, 'min_child_samples': 23, 'feature_fraction': 0.6952283228551174,

Early stopping, best iteration is:
[51]	valid_0's auc: 0.785055
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[15]	valid_0's auc: 0.744883
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[44]	valid_0's auc: 0.775307
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[7]	valid_0's auc: 0.776352


...

Did not meet early stopping. Best iteration is:
[77]	valid_0's auc: 0.861717
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[35]	valid_0's auc: 0.852268


[I 2025-08-04 22:22:22,992] Trial 27 finished with value: 0.8686914135733033 and parameters: {'learning_rate': 0.1563275809897855, 'num_leaves': 57, 'max_depth': 10, 'min_child_samples': 17, 'feature_fraction': 0.9242316736545718, 'bagging_fraction': 0.7368696212401313, 'lambda_l1': 0.7533344411206039, 'lambda_l2': 0.40436200786802456}. Best is trial 22 with value: 0.8711661042369704.


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[89]	valid_0's auc: 0.868691
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 22:22:23,147] Trial 28 finished with value: 0.8595050618672666 and parameters: {'learning_rate': 0.15530322314755615, 'num_leaves': 56, 'max_depth': 10, 'min_child_samples': 28, 'feature_fraction': 0.9184346829583186, 'bagging_fraction': 0.7319603592372081, 'lambda_l1': 0.8085446708354381, 'lambda_l2': 0.5738733137523716}. Best is trial 22 with value: 0.8711661042369704.
[I 2025-08-04 22:22:23,295] Trial 29 finished with value: 0.8398950131233596 and parameters: {'learning_rate': 0.16309696692851314, 'num_leaves': 26, 'max_depth': 12, 'min_child_samples': 24, 'feature_fraction': 0.9526650630908046, 'bagging_fraction': 0.8261099249471076, 'lambda_l1': 0.9270527388773682, 'lambda_l2': 0.4037524613429232}. Best is trial 22 with value: 0.8711661042369704.


Early stopping, best iteration is:
[62]	valid_0's auc: 0.859505
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[50]	valid_0's auc: 0.839895
✅ SR-p53 - AUC: 0.8871, Accuracy: 0.9189, Threshold: 0.11


## Results Summary

In [11]:
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="auc", ascending=False)

print("📊 Final model performance (LightGBM + XGBoost ensemble):")
display(results_df)

# Save
results_df.to_csv(os.path.join(MODEL_PATH, "training_metrics_ensemble.csv"), index=False)
joblib.dump(feature_mask_dict, os.path.join(MODEL_PATH, "feature_masks.pkl"))
joblib.dump(threshold_dict, os.path.join(MODEL_PATH, "thresholds.pkl"))

📊 Final model performance (LightGBM + XGBoost ensemble):


,label,auc,accuracy,n_train,n_val,n_test,best_threshold,num_features
10,SR-MMP,0.9400,0.9002,4648,581,581,0.15,2055
6,NR-PPAR-gamma,0.9392,0.9690,5160,645,645,0.11,2055
8,SR-ATAD5,0.9124,0.9548,5657,707,708,0.10,2055
11,SR-p53,0.8871,0.9189,5419,677,678,0.11,2055
2,NR-AhR,0.8812,0.9115,5239,655,655,0.43,2055
5,NR-ER-LBD,0.8306,0.9583,5564,695,696,0.17,2055
3,NR-Aromatase,0.8284,0.9571,4656,582,583,0.35,2055
0,NR-AR,0.8244,0.9697,5812,726,727,0.81,2055
7,SR-ARE,0.8131,0.8168,4665,583,584,0.21,2055
9,SR-HSE,0.7978,0.9413,5173,647,647,0.16,2055


['tox21_lightgb_pipeline/models/v6\\thresholds.pkl']

# A test run 

In [12]:
import joblib
import shap
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

def predict_and_explain_all_labels(smiles):
    # Compute full descriptors (1613 features)
    desc = compute_descriptors(smiles)
    if desc is None:
        raise ValueError("Invalid SMILES string.")
    
    X_input_full = np.array([list(desc.values())])
    features_all = list(desc.keys())

    # Load saved masks and thresholds
    thresholds = joblib.load(os.path.join(MODEL_PATH, "thresholds.pkl"))
    feature_masks = joblib.load(os.path.join(MODEL_PATH, "feature_masks.pkl"))

    predicted_labels = []
    all_results = {}

    for label_name in label_cols:
        # Apply label-specific feature selection
        kept_indices = feature_masks[label_name]
        X_input = X_input_full[:, kept_indices]
        features = [features_all[i] for i in kept_indices]

        # Load trained model
        model_file = os.path.join(MODEL_PATH, f"{label_name}.pkl")
        if not os.path.exists(model_file):
            print(f"⚠️ Skipping {label_name}: Model file missing")
            continue

        model = joblib.load(model_file)
        threshold = thresholds[label_name]

        # Predict probability and apply threshold
        prob = model.predict_proba(X_input)[0, 1]
        pred = int(prob >= threshold)

        if pred == 1:
            predicted_labels.append(label_name)

            # SHAP Explanation (TreeExplainer for speed)
            explainer = shap.Explainer(model)
            shap_values = explainer(X_input)

            # SHAP dataframe (sorted by abs importance)
            shap_df = pd.DataFrame({
                "feature": features,
                "shap_value": shap_values.values[0],
                "feature_value": X_input[0]
            }).sort_values(by="shap_value", key=np.abs, ascending=False)

            all_results[label_name] = {
                "prob": prob,
                "threshold": threshold,
                "shap_df": shap_df,
                "top_features": shap_df[["feature", "shap_value"]].head(4).values.tolist()
            }

    return {
        "smiles": smiles,
        "predicted_labels": predicted_labels,
        "explanations": all_results
    }


In [13]:
result = predict_and_explain_all_labels("CC(=O)Oc1ccccc1C(=O)O")  # Aspirin

⚠️ Skipping NR-AR: Model file missing
⚠️ Skipping NR-AR-LBD: Model file missing
⚠️ Skipping NR-AhR: Model file missing
⚠️ Skipping NR-Aromatase: Model file missing
⚠️ Skipping NR-ER: Model file missing
⚠️ Skipping NR-ER-LBD: Model file missing
⚠️ Skipping NR-PPAR-gamma: Model file missing
⚠️ Skipping SR-ARE: Model file missing
⚠️ Skipping SR-ATAD5: Model file missing
⚠️ Skipping SR-HSE: Model file missing
⚠️ Skipping SR-MMP: Model file missing
⚠️ Skipping SR-p53: Model file missing


# SHAP + prediction interface @@@@

In [14]:
# Core libraries
import os
import numpy as np
import pandas as pd
import joblib

# Chem + descriptor engine
from rdkit import Chem
from mordred import Calculator, descriptors

# Models
import lightgbm as lgb
import xgboost as xgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

# Explanation
import shap

# Utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.feature_selection import SelectFromModel
import matplotlib.pyplot as plt

# Optional: Optuna (used in training step)
import optuna


## compute_descriptors()

In [15]:
from mordred import Calculator, descriptors
from rdkit import Chem

# Define Mordred calculator globally
mordred_calc = Calculator(descriptors, ignore_3D=True)

def compute_descriptors(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None
        descs = mordred_calc(mol)
        descs = descs.fill_missing(0.0)  # Replace NaNs
        return descs.asdict()
    except:
        return None


## Visualize SHAP for Any Label

In [16]:
def visualize_shap_for_label(label_name, smiles):
    # Compute descriptors
    desc = compute_descriptors(smiles)
    if desc is None:
        raise ValueError("Invalid SMILES string.")
    
    X_input_full = np.array([list(desc.values())])
    features_all = list(desc.keys())

    # Load feature mask
    feature_mask = joblib.load(os.path.join(MODEL_PATH, "feature_masks.pkl"))[label_name]
    X_input = X_input_full[:, feature_mask]
    features = [features_all[i] for i in feature_mask]

    # Load model (use LightGBM for SHAP)
    model = joblib.load(os.path.join(MODEL_PATH, f"{label_name}_lgb.pkl"))

    # SHAP explanation
    explainer = shap.Explainer(model, X_input)
    shap_values = explainer(X_input)

    shap_df = pd.DataFrame({
        "feature": features,
        "shap_value": shap_values.values[0],
        "feature_value": X_input[0]
    }).sort_values(by="shap_value", key=np.abs, ascending=False)

    return shap_values, shap_df


## predict_and_explain_all_labels (smiles)

In [17]:
# Load full descriptors (length = 1613)
desc = compute_descriptors(smiles)
X_input_full = np.array([list(desc.values())])
features_all = list(desc.keys())

# Load masks and thresholds
thresholds = joblib.load(os.path.join(MODEL_PATH, "thresholds.pkl"))

for label_name in label_cols:
    # Apply correct mask for this label
    kept_indices = feature_masks[label_name]
    X_input = X_input_full[:, kept_indices]  # This will match model's input size
    features = [features_all[i] for i in kept_indices]

    model = joblib.load(os.path.join(MODEL_PATH, f"{label_name}.pkl"))

    # Predict
    prob = model.predict_proba(X_input)[0, 1]


AttributeError: 'NoneType' object has no attribute 'values'

## summarize_explanation

In [ ]:
def summarize_explanation(result_dict):
    smiles = result_dict["smiles"]
    predicted = result_dict["predicted_labels"]
    explanations = result_dict["explanations"]

    # Resolve molecule name (optional)
    try:
        from rdkit import Chem
        from rdkit.Chem import Descriptors
        mol = Chem.MolFromSmiles(smiles)
        name = Descriptors.MolWt(mol)
        name_str = f" (MolWt: {name:.1f})"
    except:
        name_str = ""

    if not predicted:
        print(f"No toxicity predicted for SMILES: `{smiles}`{name_str}")
        return

    print(f"\n🧪 **Toxicity Report** for SMILES: `{smiles}`{name_str}\n")
    print(f"✅ Predicted to be toxic in the following endpoints:\n")

    for label in predicted:
        print(f"  • {label}")

    print("\n🔍 Top SHAP features:")

    for label in predicted:
        top_feats = explanations[label]["top_features"]
        feat_text = ", ".join([f"`{f}` ({v:.3f})" for f, v in top_feats])
        print(f"  • **{label}**: influenced by {feat_text}")


## predict_explain_and_summarize

In [ ]:
def predict_explain_and_summarize(smiles):
    result = predict_and_explain_all_labels(smiles)
    summarize_explanation(result)


## batch_predict_and_explain

In [ ]:
def batch_predict_and_explain(csv_path, output_dir):
    import time
    df = pd.read_csv(csv_path)

    os.makedirs(output_dir, exist_ok=True)
    all_outputs = []

    for i, row in df.iterrows():
        smiles = row["smiles"]
        mol_id = row.get("mol_id", f"mol_{i}")
        try:
            result = predict_and_explain_all_labels(smiles)
            result["mol_id"] = mol_id

            # Save SHAP per label
            for label, details in result["explanations"].items():
                shap_df = details["shap_df"]
                shap_df["mol_id"] = mol_id
                shap_df["label"] = label
                shap_df["probability"] = details["prob"]
                shap_df["threshold"] = details["threshold"]
                shap_df.to_csv(os.path.join(output_dir, f"{mol_id}_{label}_shap.csv"), index=False)

            all_outputs.append({
                "mol_id": mol_id,
                "smiles": smiles,
                "predicted_labels": ";".join(result["predicted_labels"])
            })

        except Exception as e:
            print(f"❌ Failed on {mol_id}: {e}")

    summary_df = pd.DataFrame(all_outputs)
    summary_df.to_csv(os.path.join(output_dir, "toxicity_predictions.csv"), index=False)
    print(f"\n✅ Batch prediction completed for {len(all_outputs)} molecules.")


## running on full dataset

In [18]:

batch_predict_and_explain(
    csv_path="tox21_lightgb_pipeline/Data_v6/processed/tox21.csv",
    output_dir="tox21_lightgb_pipeline/Data_v6/SHAP"
)

NameError: name 'batch_predict_and_explain' is not defined

# All in one (attempting to add everything in one place)

## STEP 1: Data Loading & Descriptor Computation

In [1]:
import pandas as pd
import numpy as np
import os
from mordred import Calculator, descriptors
from rdkit import Chem
from tqdm import tqdm

tqdm.pandas()

# Define paths
DATA_PATH = "tox21_lightgb_pipeline/Data_v6/processed/tox21.csv"
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"
os.makedirs(SAVE_DIR, exist_ok=True)

# Load data
df = pd.read_csv(DATA_PATH)
label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

# Filter valid SMILES
df = df[df["smiles"].apply(lambda x: isinstance(x, str))]

# Compute descriptors
calc = Calculator(descriptors, ignore_3D=True)

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        return calc(mol)
    except:
        return None

mols = df["smiles"].progress_apply(Chem.MolFromSmiles)
descs = mols.progress_apply(lambda m: calc(m) if m is not None else None)
desc_df = pd.DataFrame([d.asdict() if d is not None else {} for d in descs])
desc_df = desc_df.dropna(axis=1)

# Save feature names
with open(os.path.join(SAVE_DIR, "feature_names.txt"), "w") as f:
    f.write("\n".join(desc_df.columns))

# Extract X and 
X = desc_df.values.astype(np.float32)
Y = df[label_cols].values.astype(np.float32)
label_mask = ~df[label_cols].isna().values

np.save(os.path.join(SAVE_DIR, "X.npy"), X)
np.save(os.path.join(SAVE_DIR, "Y.npy"), Y)
np.save(os.path.join(SAVE_DIR, "label_mask.npy"), label_mask)

  1%|          | 70/7831 [00:02<04:45, 27.18it/s][22:26:09] WARNING: not removing hydrogen atom without neighbors
[22:26:09] WARNING: not removing hydrogen atom without neighbors
[22:26:09] WARNING: not removing hydrogen atom without neighbors
  1%|▏         | 114/7831 [00:04<09:16, 13.87it/s]d:\Coding Projects\Predicting-Drug-Response-Using-Multi-Omics-Data-with-XAI\lord\Lib\site-packages\numpy\core\fromnumeric.py:88: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
100%|██████████| 7831/7831 [06:27<00:00, 20.22it/s]


### Handle NaN values in Y labels (classification targets)

In [11]:
import numpy as np
import os

# Reload just to be sure
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"
Y = np.load(os.path.join(SAVE_DIR, "Y.npy"))
label_mask = np.load(os.path.join(SAVE_DIR, "label_mask.npy"))

# Strategy: Replace NaN in Y with -1 (we won't train on those)
# Alternatively, you can use `label_mask` directly for filtering later
Y_cleaned = np.where(label_mask, Y, -1)

# Save cleaned Y for model training
np.save(os.path.join(SAVE_DIR, "Y.npy"), Y_cleaned)

print("✅ Y_cleaned.npy saved. NaNs replaced with -1 for masking during model training.")


✅ Y_cleaned.npy saved. NaNs replaced with -1 for masking during model training.


## STEP 2: Model Training (LightGBM)

In [ ]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_curve
import joblib
import optuna
import numpy as np
import pandas as pd
import os

# === Load precomputed data ===
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"
MODEL_PATH = "tox21_lightgb_pipeline/models/v6"
os.makedirs(MODEL_PATH, exist_ok=True)

X = np.load(os.path.join(SAVE_DIR, "X.npy"))
Y = np.load(os.path.join(SAVE_DIR, "Y.npy"))
label_mask = np.load(os.path.join(SAVE_DIR, "label_mask.npy"))
with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    feature_names = f.read().splitlines()

# === Define labels ===
label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

# === Training Setup ===
results = []
feature_mask_dict = {}
threshold_dict = {}

for i, label_name in enumerate(label_cols):
    print(f"\n📍 Training for label: {label_name}")

    # Subset valid rows for this label
    valid_rows = label_mask[:, i]
    X_valid = X[valid_rows]
    y_valid = Y[valid_rows, i]

    # Split the data
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_valid, y_valid, test_size=0.2, random_state=42, stratify=y_valid
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
    )

    # === Feature Selection ===
    selector_model = lgb.LGBMClassifier(n_estimators=200, random_state=42)
    selector_model.fit(X_train, y_train)
    selector = SelectFromModel(selector_model, threshold="median", prefit=True)
    kept_idx = selector.get_support(indices=True)

    X_train_fs = selector.transform(X_train)
    X_val_fs = selector.transform(X_val)
    X_test_fs = selector.transform(X_test)

    feature_mask_dict[label_name] = kept_idx

    # === Class Imbalance Handling ===
    n_pos = np.sum(y_train == 1)
    n_neg = np.sum(y_train == 0)
    scale_pos_weight = n_neg / n_pos if n_pos > 0 else 1.0

    # === Hyperparameter Tuning with Optuna ===
    dtrain = lgb.Dataset(X_train_fs, label=y_train)
    dval = lgb.Dataset(X_val_fs, label=y_val)

    def objective(trial):
        params = {
            "objective": "binary",
            "metric": "auc",
            "boosting_type": "gbdt",
            "verbosity": -1,
            "feature_pre_filter": False,
            "scale_pos_weight": scale_pos_weight,
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
            "num_leaves": trial.suggest_int("num_leaves", 20, 80),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
            "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 1.0),
            "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 1.0),
        }
        model = lgb.train(
            params, dtrain, valid_sets=[dval], callbacks=[
                early_stopping(stopping_rounds=30),
                log_evaluation(period=0)
            ]
        )
        preds = model.predict(X_val_fs)
        return roc_auc_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    best_params = study.best_params
    best_params.update({
        "objective": "binary",
        "metric": "auc",
        "boosting_type": "gbdt",
        "verbosity": -1,
        "scale_pos_weight": scale_pos_weight
    })

    # === Final Model Training ===
    final_model = lgb.LGBMClassifier(**best_params, n_estimators=500)
    final_model.fit(X_train_fs, y_train)

    # === Threshold Selection ===
    preds_val = final_model.predict_proba(X_val_fs)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_val, preds_val)
    f1_scores = 2 * precision * recall / (precision + recall + 1e-8)
    best_idx = np.argmax(f1_scores)
    best_thresh = thresholds[best_idx]

    # === Final Evaluation ===
    y_pred_bin = (final_model.predict_proba(X_test_fs)[:, 1] >= best_thresh).astype(int)
    auc = roc_auc_score(y_test, final_model.predict_proba(X_test_fs)[:, 1])
    acc = accuracy_score(y_test, y_pred_bin)

    # === Save Artifacts ===
    joblib.dump(final_model, os.path.join(MODEL_PATH, f"{label_name}.pkl"))
    threshold_dict[label_name] = best_thresh

    results.append({
        "label": label_name,
        "auc": auc,
        "accuracy": acc,
        "n_train": len(y_train),
        "n_val": len(y_val),
        "n_test": len(y_test),
        "best_threshold": best_thresh,
        "num_features": len(kept_idx),
        "scale_pos_weight": scale_pos_weight
    })

# === Save All Utilities ===
joblib.dump(threshold_dict, os.path.join(MODEL_PATH, "thresholds.pkl"))
joblib.dump(feature_mask_dict, os.path.join(MODEL_PATH, "feature_masks.pkl"))
pd.DataFrame(results).to_csv(os.path.join(MODEL_PATH, "training_metrics.csv"), index=False)


📍 Training for label: NR-AR


[I 2025-08-04 22:45:42,281] A new study created in memory with name: no-name-48feed37-b7c1-4dc8-8321-f0259a752ea9
[I 2025-08-04 22:45:42,690] Trial 0 finished with value: 0.6950104432582965 and parameters: {'learning_rate': 0.12548983540874797, 'num_leaves': 61, 'max_depth': 3, 'min_child_samples': 27, 'feature_fraction': 0.81759911359066, 'bagging_fraction': 0.8221146941074708, 'lambda_l1': 0.0699981664681033, 'lambda_l2': 0.36492819595499515}. Best is trial 0 with value: 0.6950104432582965.


Training until validation scores don't improve for 30 rounds
Did not meet early stopping. Best iteration is:
[82]	valid_0's auc: 0.69501
Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[11]	valid_0's auc: 0.727942


[I 2025-08-04 22:45:42,947] Trial 1 finished with value: 0.7279415177535391 and parameters: {'learning_rate': 0.08230155003401189, 'num_leaves': 31, 'max_depth': 7, 'min_child_samples': 29, 'feature_fraction': 0.6131242838472909, 'bagging_fraction': 0.8964264489053455, 'lambda_l1': 0.28480761237641183, 'lambda_l2': 0.3633740275928765}. Best is trial 1 with value: 0.7279415177535391.


Training until validation scores don't improve for 30 rounds
Early stopping, best iteration is:
[3]	valid_0's auc: 0.723091


...

Early stopping, best iteration is:
[62]	valid_0's auc: 0.885714
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 22:49:34,716] Trial 28 finished with value: 0.8673790776152981 and parameters: {'learning_rate': 0.14210232297837835, 'num_leaves': 40, 'max_depth': 7, 'min_child_samples': 44, 'feature_fraction': 0.8006665592646889, 'bagging_fraction': 0.8337601916691072, 'lambda_l1': 0.8985050093666247, 'lambda_l2': 0.7967267440386506}. Best is trial 24 with value: 0.9016497937757781.


Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.867379
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 22:49:35,034] Trial 29 finished with value: 0.9050618672665917 and parameters: {'learning_rate': 0.1201703155829826, 'num_leaves': 34, 'max_depth': 5, 'min_child_samples': 33, 'feature_fraction': 0.9242927584196634, 'bagging_fraction': 0.6896252731160479, 'lambda_l1': 0.7960188286042771, 'lambda_l2': 0.8896506764156475}. Best is trial 29 with value: 0.9050618672665917.


Early stopping, best iteration is:
[40]	valid_0's auc: 0.905062


### 2.1 eval 

In [12]:
import pandas as pd
import os

# Create results DataFrame
results_df = pd.DataFrame(results)

# Sort by AUC (optional)
results_df = results_df.sort_values(by="auc", ascending=False)

# Display results
print("📈 Summary of LightGBM model performance:")
display(results_df)

# Save to CSV
results_df.to_csv(os.path.join(MODEL_PATH, "training_metrics.csv"), index=False)


📈 Summary of LightGBM model performance:


,label,auc,accuracy,n_train,n_val,n_test,best_threshold,num_features,scale_pos_weight
10,SR-MMP,0.948920,0.924269,4648,581,581,0.344759,812,5.332425
8,SR-ATAD5,0.886605,0.951977,5657,707,708,0.140595,931,25.810427
2,NR-AhR,0.881095,0.893130,5239,655,655,0.487913,807,7.532573
6,NR-PPAR-gamma,0.877636,0.964341,5160,645,645,0.653218,907,33.630872
11,SR-p53,0.858378,0.920354,5419,677,678,0.134669,828,15.032544
9,SR-HSE,0.853877,0.927357,5173,647,647,0.081956,819,16.359060
7,SR-ARE,0.832675,0.835616,4665,583,584,0.211113,831,5.187003
5,NR-ER-LBD,0.822563,0.965517,5564,695,696,0.657139,964,18.871429
3,NR-Aromatase,0.822122,0.953688,4656,582,583,0.373381,836,18.400000
0,NR-AR,0.813450,0.971114,5812,726,727,0.978191,943,22.530364


## 2*: Train LightGBM Models Using All Features (No Feature Selection)

In [1]:
import os
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, precision_recall_curve
import optuna
import joblib

X = np.load("tox21_lightgb_pipeline/Data_v6/processed/X.npy")
Y = np.load("tox21_lightgb_pipeline/Data_v6/processed/Y.npy")
label_mask = np.load("tox21_lightgb_pipeline/Data_v6/processed/label_mask.npy")

with open("tox21_lightgb_pipeline/Data_v6/processed/feature_names.txt") as f:
    feature_names = f.read().splitlines()

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

MODEL_PATH = "tox21_lightgb_pipeline/models/v6"
os.makedirs(MODEL_PATH, exist_ok=True)

threshold_dict = {}

for i, label_name in enumerate(label_cols):
    print(f"📍 Training for label: {label_name}")

    valid_rows = label_mask[:, i]
    X_valid = X[valid_rows]
    y_valid = Y[valid_rows, i]

    X_train, X_temp, y_train, y_temp = train_test_split(
        X_valid, y_valid, test_size=0.2, stratify=y_valid, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
    )

    dtrain = lgb.Dataset(X_train, label=y_train)
    dval = lgb.Dataset(X_val, label=y_val)

    def objective(trial):
        params = {
            "objective": "binary",
            "metric": "auc",
            "boosting_type": "gbdt",
            "verbosity": -1,
            "feature_pre_filter": False,
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
            "num_leaves": trial.suggest_int("num_leaves", 20, 80),
            "max_depth": trial.suggest_int("max_depth", 3, 12),
            "min_child_samples": trial.suggest_int("min_child_samples", 10, 50),
            "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
            "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
            "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 1.0),
            "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 1.0),
            "scale_pos_weight": sum(y_train == 0) / sum(y_train == 1),
        }
        model = lgb.train(
            params,
            dtrain,
            valid_sets=[dval],
            callbacks=[lgb.early_stopping(stopping_rounds=30), lgb.log_evaluation(period=0)]
        )
        preds = model.predict(X_val)
        return roc_auc_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    best_params = study.best_params
    best_params.update({
        "objective": "binary", "metric": "auc", "verbosity": -1, "boosting_type": "gbdt"
    })

    final_model = lgb.LGBMClassifier(**best_params, n_estimators=500)
    final_model.fit(X_train, y_train)

    # Save model
    joblib.dump(final_model, os.path.join(MODEL_PATH, f"{label_name}.pkl"))

    # Compute and save threshold
    preds_val = final_model.predict_proba(X_val)[:, 1]
    precision, recall, thresholds = precision_recall_curve(y_val, preds_val)
    f1_scores = 2 * precision * recall / (precision + recall + 1e-8)
    best_thresh = thresholds[np.argmax(f1_scores)]
    threshold_dict[label_name] = best_thresh

# Save thresholds
joblib.dump(threshold_dict, os.path.join(MODEL_PATH, "thresholds.pkl"))


[I 2025-08-04 23:00:49,680] A new study created in memory with name: no-name-e51e3d81-02cb-4a3a-b48a-d9759f256712


📍 Training for label: NR-AR
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 23:00:50,599] Trial 0 finished with value: 0.7320491993501973 and parameters: {'learning_rate': 0.13402338975175815, 'num_leaves': 47, 'max_depth': 9, 'min_child_samples': 42, 'feature_fraction': 0.9096214406960866, 'bagging_fraction': 0.7371821929763615, 'lambda_l1': 0.9723280700534054, 'lambda_l2': 0.21795035154360598}. Best is trial 0 with value: 0.7320491993501973.


Early stopping, best iteration is:
[18]	valid_0's auc: 0.732049
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 23:00:50,879] Trial 1 finished with value: 0.7464376885588304 and parameters: {'learning_rate': 0.19423906055200094, 'num_leaves': 50, 'max_depth': 4, 'min_child_samples': 13, 'feature_fraction': 0.8470602632442801, 'bagging_fraction': 0.7447417106829443, 'lambda_l1': 0.52595586613831, 'lambda_l2': 0.25784463525149903}. Best is trial 1 with value: 0.7464376885588304.


...

[I 2025-08-04 23:05:51,982] Trial 28 finished with value: 0.8716535433070867 and parameters: {'learning_rate': 0.10914922801102256, 'num_leaves': 76, 'max_depth': 11, 'min_child_samples': 27, 'feature_fraction': 0.7779255804671497, 'bagging_fraction': 0.9915533909843324, 'lambda_l1': 0.6978004546415254, 'lambda_l2': 0.25326232752216093}. Best is trial 17 with value: 0.8999250093738282.


Did not meet early stopping. Best iteration is:
[96]	valid_0's auc: 0.871654
Training until validation scores don't improve for 30 rounds


[I 2025-08-04 23:05:52,590] Trial 29 finished with value: 0.8873265841769779 and parameters: {'learning_rate': 0.18647945193848248, 'num_leaves': 80, 'max_depth': 8, 'min_child_samples': 37, 'feature_fraction': 0.6190211184784666, 'bagging_fraction': 0.9974407664668596, 'lambda_l1': 0.44248548131837645, 'lambda_l2': 0.436063766727822}. Best is trial 17 with value: 0.8999250093738282.


Early stopping, best iteration is:
[25]	valid_0's auc: 0.887327


['tox21_lightgb_pipeline/models/v6\\thresholds.pkl']

In [3]:
import numpy as np
import pandas as pd
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score

# Load saved data
X = np.load("tox21_lightgb_pipeline/Data_v6/processed/X.npy")
Y = np.load("tox21_lightgb_pipeline/Data_v6/processed/Y.npy")
label_mask = np.load("tox21_lightgb_pipeline/Data_v6/processed/label_mask.npy")

MODEL_PATH = "tox21_lightgb_pipeline/models/v6"
thresholds = joblib.load(os.path.join(MODEL_PATH, "thresholds.pkl"))

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

eval_results = []

for i, label_name in enumerate(label_cols):
    print(f"📊 Evaluating: {label_name}")
    
    valid_rows = label_mask[:, i]
    X_valid = X[valid_rows]
    y_valid = Y[valid_rows, i]

    # Same train/val/test split as training
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_valid, y_valid, test_size=0.2, stratify=y_valid, random_state=42
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42
    )

    model = joblib.load(os.path.join(MODEL_PATH, f"{label_name}.pkl"))
    threshold = thresholds[label_name]

    prob_test = model.predict_proba(X_test)[:, 1]
    pred_test = (prob_test >= threshold).astype(int)

    auc = roc_auc_score(y_test, prob_test)
    acc = accuracy_score(y_test, pred_test)
    f1 = f1_score(y_test, pred_test)

    eval_results.append({
        "label": label_name,
        "test_auc": round(auc, 4),
        "test_accuracy": round(acc, 4),
        "test_f1": round(f1, 4),
        "n_test": len(y_test)
    })

# Save evaluation results
eval_df = pd.DataFrame(eval_results)
eval_df = eval_df.sort_values(by="test_auc", ascending=False)

print("\n✅ Evaluation Summary:")
display(eval_df)

eval_df.to_csv(os.path.join(MODEL_PATH, "evaluation_metrics.csv"), index=False)


📊 Evaluating: NR-AR
📊 Evaluating: NR-AR-LBD
📊 Evaluating: NR-AhR
📊 Evaluating: NR-Aromatase
📊 Evaluating: NR-ER
📊 Evaluating: NR-ER-LBD
📊 Evaluating: NR-PPAR-gamma
📊 Evaluating: SR-ARE
📊 Evaluating: SR-ATAD5
📊 Evaluating: SR-HSE
📊 Evaluating: SR-MMP
📊 Evaluating: SR-p53

✅ Evaluation Summary:


,label,test_auc,test_accuracy,test_f1,n_test
10,SR-MMP,0.9484,0.9329,0.7821,581
6,NR-PPAR-gamma,0.9339,0.9736,0.3200,645
8,SR-ATAD5,0.9142,0.9576,0.5000,708
2,NR-AhR,0.8938,0.9069,0.5734,655
11,SR-p53,0.8587,0.9041,0.3564,678
9,SR-HSE,0.8505,0.9413,0.4242,647
5,NR-ER-LBD,0.8483,0.9569,0.5312,696
7,SR-ARE,0.8343,0.8493,0.5368,584
3,NR-Aromatase,0.8318,0.9400,0.4068,583
0,NR-AR,0.8266,0.9684,0.5490,727


## step 3: Predict + SHAP Explanation

In [15]:
import shap
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors as rdmd

from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import MolFromSmiles
from rdkit.Chem.rdMolDescriptors import CalcMolFormula

from rdkit.Chem import MolStandardize


MODEL_PATH = 'tox21_lightgb_pipeline/models/v_base'

import warnings
warnings.filterwarnings("ignore")

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        return calc(mol).asdict()
    except:
        return None

feature_masks = joblib.load(os.path.join(MODEL_PATH, "feature_masks.pkl"))
thresholds = joblib.load(os.path.join(MODEL_PATH, "thresholds.pkl"))

with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    full_features = f.read().splitlines()


def predict_and_explain_all_labels(smiles):
    desc = compute_descriptors(smiles)
    if desc is None:
        raise ValueError("Invalid SMILES")

    X_input_full = np.array([list(desc.values())])
    features_all = list(desc.keys())

    all_results = {}
    predicted_labels = []

    for label_name in label_cols:
        kept_indices = feature_masks[label_name]
        X_input = X_input_full[:, kept_indices]
        features = [features_all[i] for i in kept_indices]

        model = joblib.load(os.path.join(MODEL_PATH, f"{label_name}.pkl"))
        threshold = thresholds[label_name]
        prob = model.predict_proba(X_input)[0, 1]
        pred = int(prob >= threshold)

        if pred == 1:
            predicted_labels.append(label_name)
            explainer = shap.Explainer(model)
            shap_values = explainer(X_input)

            shap_df = pd.DataFrame({
                "feature": features,
                "shap_value": shap_values.values[0],
                "feature_value": X_input[0]
            }).sort_values(by="shap_value", key=np.abs, ascending=False)

            all_results[label_name] = {
                "prob": prob,
                "threshold": threshold,
                "shap_df": shap_df,
                "top_features": shap_df[["feature", "shap_value"]].head(4).values.tolist()
            }

    return {
        "smiles": smiles,
        "predicted_labels": predicted_labels,
        "explanations": all_results
    }

## step 3*: Explain Using All Features

In [18]:
import shap
import numpy as np
import pandas as pd
import joblib
from rdkit import Chem
from mordred import Calculator, descriptors

MODEL_PATH = "tox21_lightgb_pipeline/models/v6"
SAVE_DIR = "tox21_lightgb_pipeline/Data_v6/processed"

# Load features
with open(os.path.join(SAVE_DIR, "feature_names.txt")) as f:
    full_features = f.read().splitlines()

thresholds = joblib.load(os.path.join(MODEL_PATH, "thresholds.pkl"))

calc = Calculator(descriptors, ignore_3D=True)

def compute_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None
    try:
        return calc(mol).asdict()
    except:
        return None

label_cols = [
    "NR-AR", "NR-AR-LBD", "NR-AhR", "NR-Aromatase", "NR-ER",
    "NR-ER-LBD", "NR-PPAR-gamma", "SR-ARE", "SR-ATAD5",
    "SR-HSE", "SR-MMP", "SR-p53"
]

def predict_and_explain_all_labels(smiles):
    desc = compute_descriptors(smiles)
    if desc is None:
        raise ValueError("Invalid SMILES")

    X_input = np.array([list(desc.values())])
    features = list(desc.keys())

    results = {}
    predicted_labels = []

    for label_name in label_cols:
        model = joblib.load(os.path.join(MODEL_PATH, f"{label_name}.pkl"))
        threshold = thresholds[label_name]

        prob = model.predict_proba(X_input)[0, 1]
        pred = int(prob >= threshold)

        if pred == 1:
            predicted_labels.append(label_name)

            explainer = shap.Explainer(model)
            shap_values = explainer(X_input)

            shap_df = pd.DataFrame({
                "feature": features,
                "shap_value": shap_values.values[0],
                "feature_value": X_input[0]
            }).sort_values(by="shap_value", key=np.abs, ascending=False)

            results[label_name] = {
                "prob": prob,
                "threshold": threshold,
                "shap_df": shap_df,
                "top_features": shap_df[["feature", "shap_value"]].head(4).values.tolist()
            }

    return {
        "smiles": smiles,
        "predicted_labels": predicted_labels,
        "explanations": results
    }


## step4: Summarise Explanation

In [19]:
def summarize_prediction(result):
    smiles = result["smiles"]
    predicted = result["predicted_labels"]
    if not predicted:
        print(f"The drug (SMILES: `{smiles}`) is predicted to have no significant toxicity endpoints.")
        return

    text = f"The given drug (SMILES: `{smiles}`) is predicted to be toxic in: **{', '.join(predicted)}**.\n"
    for label in predicted:
        feats = result["explanations"][label]["top_features"]
        feat_str = ", ".join([f"`{f[0]}` ({f[1]:.2f})" for f in feats])
        text += f"\n→ For **{label}**, important features include: {feat_str}"
    print(text)

In [20]:
# Example usage
result = predict_and_explain_all_labels("O=C(O)Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1")  # Aspirin
summarize_prediction(result)


The given drug (SMILES: `O=C(O)Cc1cc(I)c(Oc2ccc(O)c(I)c2)c(I)c1`) is predicted to be toxic in: **NR-ER, SR-HSE**.

→ For **NR-ER**, important features include: `BIC5` (0.32), `RPCG` (0.30), `piPC10` (0.28), `AATS8i` (0.27)
→ For **SR-HSE**, important features include: `SLogP` (0.76), `AATSC2s` (-0.32), `SsOH` (0.28), `AATS0i` (0.26)
